# Conexión de Spark a una base de datos relacional (RDBMS)

Para leer datos desde una base de datos relacional (RDBMS) usando Spark, puedes utilizar el conector JDBC. Spark permite conectarse a bases de datos como MySQL, PostgreSQL, SQL Server, entre otras, mediante el uso de la URL JDBC correspondiente y las credenciales de acceso.

A continuación se muestra un ejemplo básico de cómo leer una tabla desde una RDBMS usando Spark DataFrame:

```python

jdbc_url = "jdbc:mysql://<host>:<puerto>/<base_de_datos>"
table = "<nombre_de_tabla>"
properties = {
    "user": "<usuario>",
    "password": "<contraseña>"
}

df = spark.read.jdbc(url=jdbc_url, table=table, properties=properties)
display(df)
```

**Pasos principales:**
1. Define la URL JDBC de la base de datos.
2. Especifica el nombre de la tabla que deseas leer.
3. Proporciona las credenciales y el driver JDBC adecuado.
4. Utiliza `spark.read.jdbc()` para cargar los datos en un DataFrame de Spark.
5. Usa `display(df)` para visualizar los datos en Databricks.

Recuerda que debes tener el driver JDBC correspondiente instalado en tu entorno de Databricks.

# Databricks Secret Management: Uso de Scopes y dbutils

Databricks proporciona un sistema seguro para almacenar y gestionar información sensible, como contraseñas, claves de API y credenciales, mediante el uso de **secretos**. Estos secretos se organizan en **scopes** (ámbitos), que agrupan y controlan el acceso a los secretos.

## ¿Qué es un Scope?

Un **scope** es una colección de secretos identificada por un nombre. Los scopes permiten gestionar permisos y organizar los secretos según roles, aplicaciones o equipos. Por ejemplo, puedes tener un scope llamado `jdbc` para almacenar credenciales de bases de datos.

## ¿Para qué sirven los Secretos en Databricks?

Los secretos permiten:
- Proteger información sensible evitando exponerla en el código.
- Controlar el acceso a credenciales mediante permisos.
- Facilitar la rotación y gestión centralizada de claves y contraseñas.

## ¿Cómo crear un Scope y un Secreto?

1. **Crear un scope** (desde la CLI o API):

```bash
databricks secrets create-scope mysql-secrets -p [profile]
```


2. **Crear un secreto dentro del scope**:

```bash
databricks secrets put-secret --json '{"scope":"mysql-secrets","key":"db_password","string_value":"TuPasswordSeguro123"}' -p [profile]
```

3. **Comandos utiles**

```python
# Listar todos los scopes disponibles
dbutils.secrets.listScopes()

# Listar las claves dentro de un scope
dbutils.secrets.list("mysql-secrets")

# Leer el valor de un secreto
db_password = dbutils.secrets.get("mysql-secrets", "db_password")
```

## ¿Cómo recuperar un secreto con dbutils?

En un notebook de Databricks, puedes acceder a los secretos usando `dbutils.secrets.get()`:

```python
# Recuperar el secreto 'password' del scope 'jdbc'
password = dbutils.secrets.get(scope="jdbc", key="password")
```

> **Nota:** Si intentas mostrar el valor del secreto, Databricks lo reemplazará por `[REDACTED]` para evitar filtraciones accidentales.

## Resumen

- Los secretos en Databricks se agrupan en scopes para una gestión segura y centralizada.
- Puedes crear y gestionar secretos usando la CLI, API o SDK.
- Recupera los secretos en tus notebooks con `dbutils.secrets.get()` para usarlos de forma segura en tus aplicaciones y flujos de trabajo.

# Instalación y uso del cliente de Databricks

El cliente de Databricks (Databricks CLI) es una herramienta de línea de comandos que permite interactuar con los recursos de Databricks desde tu terminal o scripts automatizados. Facilita tareas como la gestión de clústeres, trabajos, archivos y otros recursos de Databricks sin necesidad de usar la interfaz web.

## ¿Para qué sirve el cliente de Databricks?

- Automatizar tareas administrativas y de desarrollo en Databricks.
- Gestionar clústeres, trabajos, notebooks, archivos y otros recursos.
- Integrar Databricks en flujos de trabajo de CI/CD y scripts personalizados.

## Instalación del cliente de Databricks

Para instalar el cliente de Databricks, sigue estos pasos:

1. **Instala el paquete usando pip:**

```bash
%pip install databricks-cli
```
   

2. **Verifica la instalación:**

```bash
databricks --version
```
   

3. **Configura el cliente con tus credenciales:**

   Ejecuta el siguiente comando y sigue las instrucciones para ingresar tu host y token de acceso personal:

```bash
databricks configure --token
```
   

Para más detalles, consulta la [documentación oficial](https://docs.databricks.com/aws/en/dev-tools/cli/install#gsc.tab=0).

In [0]:
db_password = dbutils.secrets.get("mysql-secrets", "db_password")
print(db_password)

In [0]:
db_name = "fake"
db_user="curso"
table_name = "customers"
db_host = "www.bigdataybi.com"
db_port = 3306
jdbc_url = f"jdbc:mysql://{db_host}:{db_port}/{db_name}"
df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .option("user", db_user) \
    .option("password", db_password) \
    .load()
display(df.limit(3))

## Ejercicio práctico:

Lee las tablas:
- products
- sales
- shops

Todas ellas se encuentran en la base de datos llamada fake dentro del servidor de Mysql